In [1]:
from tabula import read_pdf
from tabula import wrapper
import pandas as pd
import pyodbc
import datetime
from datetime import timedelta
from datetime import date

import requests
import lxml.html as lh
from bs4 import BeautifulSoup
import csv
import urllib.request as urllib2


### read the pdfs and convert them in to dfs with 3 columns

In [2]:
#run this if it's on a Monday
file_date = str(date.today() - datetime.timedelta(days=3)).replace('-','_')
todays_date = str(date.today())
#if other days
# file_date = '2019_09_06'  
# todays_date = '2019-09-09' 

list_of_zones = ['JCB','JCD','NJO','NJY','INB','IND','NEB','ERN','BGY','BGZ','PCP','PCO','PPF','PPH','PSA','PSB']

#sometimes there are only 2 pages
#page_number = [1,2,3]

page_number = [1,2]
d={}
for zone in list_of_zones:
    df_zone = pd.DataFrame() 
    for page in page_number:
        df = wrapper.read_pdf('C:/Users/yangsh6/Desktop/Monthly_F_price/' + zone +'/' + zone + '_' + file_date + '.pdf',pages=page )
        for i in range(0,len(df)):               ## in this case the package is reading all the headings, need to detect where is the        
            if df.iloc[i][0] != zone:            # first row of data and read from there
                continue       
            else:
                index = i
                df = df[index: ].reset_index(drop = True)
                break
        else:
            break
        
        df = df.iloc[:,[0,1,5]]

        df.columns = ['Zone_Ctrct','Contract_M_Ctrct','Price_Ctrct']
        df.Price_Ctrct = df.Price_Ctrct.str.split(' ').str[0] 
#         df = df[0:13]                       # 12 month
        df = df[['Contract_M_Ctrct','Zone_Ctrct','Price_Ctrct']]        # move month to the first column
#         df['Record_Date_Ctrct'] = file_date.replace('_','-')
#         df['Record_Date_Ctrct'] = pd.to_datetime(df['Record_Date_Ctrct'])
        df['Record_Date_Ctrct'] = todays_date
        df['Website_Ctrct'] = 'ice'
        df['Filename_Ctrct'] = zone + '_' + file_date
        
        df.drop(df.tail(1).index,inplace=True)
        
        df_zone = df_zone.append(df)
    
    df_zone = df_zone.reset_index(drop=True)
        
    d["{}".format(zone)]=df_zone

print(d)                                 

{'JCB':    Contract_M_Ctrct Zone_Ctrct Price_Ctrct Record_Date_Ctrct Website_Ctrct  \
0             Oct19        JCB       22.64        2019-10-21           ice   
1             Nov19        JCB       24.80        2019-10-21           ice   
2             Dec19        JCB       36.95        2019-10-21           ice   
3             Jan20        JCB       61.10        2019-10-21           ice   
4             Feb20        JCB       56.30        2019-10-21           ice   
5             Mar20        JCB       33.85        2019-10-21           ice   
6             Apr20        JCB       26.80        2019-10-21           ice   
7             May20        JCB       26.25        2019-10-21           ice   
8             Jun20        JCB       25.60        2019-10-21           ice   
9             Jul20        JCB       32.95        2019-10-21           ice   
10            Aug20        JCB       29.80        2019-10-21           ice   
11            Sep20        JCB       26.00        2019-1

### sometimes the formats are messed up, so check before uploadingthem in to the server

In [3]:
# if it's hard to fix in a loop, read the zone individually and update the dictionary

# d.update({'ERN': df})
# d

### push everything in the server

In [4]:
from sqlalchemy import create_engine, event
engine = create_engine("mssql+pyodbc://:@PROD_NEW")

In [5]:
for key, value in d.items():
    to_push = value
    to_push.astype('str').to_sql('EP_contract_archive', engine, if_exists='append', chunksize=50, index= None,schema='SCMAnalyticsDev.dbo')


In [1]:
# if the results are save in df

# to_push = df
# to_push.astype('str').to_sql('EP_contract_archive', engine, if_exists='append', chunksize=50, index= None)

### fetch and push the henry gas price

In [16]:
gas = wrapper.read_pdf('C:/Users/yangsh6/Desktop/Monthly_F_price/Henry hub/H_' + file_date + '.pdf')
gas

,H,Nov19,2.321,2.362,2.290,2.362.1,2.320,0.002,"116,808","303,170",-780,0,0.1,346,"28,443"
0,H,Dec19,2.520,2.548,2.489,2.541,2.517,0.001,"43,937","326,751","-2,760",0,0,71,"30,047"
1,H,Jan20,2.646,2.665,2.608,2.659,2.635,0.002,"33,755","316,347",364,0,0,784,"23,472"
2,H,Feb20,2.616,2.627,2.575,2.627,2.603,0.005,"16,149","198,472",-505,0,0,475,"10,382"
3,H,Mar20,2.490,2.520,2.483,2.520,2.499,0.000,"18,340","247,412","1,292",0,0,505,"12,276"
4,H,Apr20,2.267,2.280,2.260,2.280,2.274,0.005,"13,547","228,395",34,0,0,778,"9,632"
5,H,May20,2.246,2.265,2.246,2.265,2.260,0.004,"7,820","221,058","4,400",0,0,"4,336",744
6,H,Jun20,2.292,2.307,2.292,2.307,2.302,0.003,"3,225","166,875",308,0,0,326,630
7,H,Jul20,2.340,2.355,2.340,2.355,2.350,0.002,"2,950","176,661",410,0,0,336,713
8,H,Aug20,2.350,2.367,2.350,2.367,2.360,0.001,"2,768","173,254",93,0,0,788,607
9,H,Sep20,2.340,2.352,2.333,2.350,2.344,0.001,"2,517","169,546",127,0,0,326,390


In [17]:
# have to check the format sometimes
gas = gas.iloc[:,[1,6]]
gas

,Nov19,2.320
0,Dec19,2.517
1,Jan20,2.635
2,Feb20,2.603
3,Mar20,2.499
4,Apr20,2.274
5,May20,2.260
6,Jun20,2.302
7,Jul20,2.350
8,Aug20,2.360
9,Sep20,2.344


In [8]:
# gas = gas[4:]
# gas

In [18]:
# the first row is read into the header
gas.loc[-1] = gas.columns                  # adding the misread header
gas.index = gas.index + 1                  # shifting index
gas = gas.sort_index()    

C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
gas.columns = ['date', 'HenryHub']
gas

,date,HenryHub
0,Nov19,2.320
1,Dec19,2.517
2,Jan20,2.635
3,Feb20,2.603
4,Mar20,2.499
5,Apr20,2.274
6,May20,2.26
7,Jun20,2.302
8,Jul20,2.35
9,Aug20,2.36


In [11]:
gas.date.str[3:5]

4     20
5     20
6     20
7     20
8     20
9     20
10    20
11    20
12    20
13    21
14    21
15    21
16    21
17    21
18    21
19    21
20    21
21    21
22    21
23    21
Name: date, dtype: object

In [20]:
# sometimes the frommats are differnt
gas.date = gas.date.str[3:5] + '-' + gas.date.str[0:3] 

# gas.HenryHub = gas.HenryHub.str.split(' ').str[0]
gas = gas.reset_index(drop=True)[:12]

In [11]:
# gas.date = gas.date.str[5:] + '-' + gas.date.str[2:5] 

# gas.HenryHub = gas.HenryHub.str.split(' ').str[0]
# gas = gas.reset_index(drop=True)[:12]

In [14]:
# gas.HenryHub = gas.HenryHub.str.split(' ').str[0]

In [21]:
gas

,date,HenryHub
0,19-Nov,2.320
1,19-Dec,2.517
2,20-Jan,2.635
3,20-Feb,2.603
4,20-Mar,2.499
5,20-Apr,2.274
6,20-May,2.26
7,20-Jun,2.302
8,20-Jul,2.35
9,20-Aug,2.36


In [22]:
gas['record_date'] = todays_date
gas['website'] = 'ice'
gas['file_name'] = 'H_' + file_date

In [23]:
gas

,date,HenryHub,record_date,website,file_name
0,19-Nov,2.320,2019-10-21,ice,H_2019_10_18
1,19-Dec,2.517,2019-10-21,ice,H_2019_10_18
2,20-Jan,2.635,2019-10-21,ice,H_2019_10_18
3,20-Feb,2.603,2019-10-21,ice,H_2019_10_18
4,20-Mar,2.499,2019-10-21,ice,H_2019_10_18
5,20-Apr,2.274,2019-10-21,ice,H_2019_10_18
6,20-May,2.26,2019-10-21,ice,H_2019_10_18
7,20-Jun,2.302,2019-10-21,ice,H_2019_10_18
8,20-Jul,2.35,2019-10-21,ice,H_2019_10_18
9,20-Aug,2.36,2019-10-21,ice,H_2019_10_18


In [24]:
# get gas from the website

## need to switch to vzinet 

url = "https://ycharts.com/indicators/natural_gas_spot_price"

page = urllib2.urlopen(url).read()

soup = BeautifulSoup(page)
table1 = soup.findAll("table")[0]


## need to switch back to Vteam to run this block

result_date = []
result_price = []
rename_months_v2 = {'Jan': 1,'Feb': 2,'Mar': 3,'Apr': 4,'May': 5,'Jun': 6,
                    'Jul': 7,'Aug': 8,'Sep': 9,'Oct': 10,'Nov': 11,'Dec': 12} 

for table_row in table1.findAll('tr')[1:]:
    date = table_row.findAll('td')[0].text.split(' ')
    m = str(rename_months_v2[date[0][:3]])
    d = str(date[1].replace(',',''))
    y = str(date[2])
    new_date = y + '-' +  m  + '-' + d
    price = table_row.findAll('td')[1].text[17:21]
    result_date.append(new_date)
    result_price.append(float(price))
    
    
result = pd.DataFrame()
result['date'] = result_date
result['gas'] =result_price
result['date'] = pd.to_datetime(result['date'])


result = result.set_index('date',drop=False)[:5]
ave = result['gas'].mean()
ave

C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


2.2139999999999995

In [25]:
##look up gas price here from https://ycharts.com/indicators/natural_gas_spot_price
from datetime import date
rec_date = todays_date[2:4] + '-' + [abb for abb, number in rename_months_v2.items() if number == date.today().month][0]

gas.loc[-1]=[rec_date,ave, todays_date,'ave','NA']
gas.index = gas.index + 1

In [26]:
gas = gas.sort_index() 
gas

,date,HenryHub,record_date,website,file_name
0,19-Oct,2.214,2019-10-21,ave,NA
1,19-Nov,2.320,2019-10-21,ice,H_2019_10_18
2,19-Dec,2.517,2019-10-21,ice,H_2019_10_18
3,20-Jan,2.635,2019-10-21,ice,H_2019_10_18
4,20-Feb,2.603,2019-10-21,ice,H_2019_10_18
5,20-Mar,2.499,2019-10-21,ice,H_2019_10_18
6,20-Apr,2.274,2019-10-21,ice,H_2019_10_18
7,20-May,2.26,2019-10-21,ice,H_2019_10_18
8,20-Jun,2.302,2019-10-21,ice,H_2019_10_18
9,20-Jul,2.35,2019-10-21,ice,H_2019_10_18


### this has to be in the prod_new too, have to migrate the table

In [31]:
from sqlalchemy import create_engine, event
engine = create_engine("mssql+pyodbc://:@PROD_NEW")

In [32]:
to_push = gas
# to_push.astype('str').to_sql('EP_HenryHub_archive', engine, if_exists='append', chunksize=50, index= None)
to_push.astype('str').to_sql('EP_HenryHub_archive', engine, if_exists='append', chunksize=50, index= None,schema='SCMAnalyticsDev.dbo')

### read the html from https://longforecast.com/oil-price-today-forecast-2017-2018-2019-2020-2021-brent-wti#1

In [27]:
# need to use the vzinet to run this cell

url = "https://longforecast.com/oil-price-today-forecast-2017-2018-2019-2020-2021-brent-wti#1"

page = urllib2.urlopen(url).read()

soup = BeautifulSoup(page)
table = soup.findAll("table")[6] # the table we need is the 7th table in the html

C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [28]:
MoY = []
Close = []

for table_row in table.findAll('tr')[1:]:
    if len(table_row) == 1 :
        year = table_row.find('td').text[2:4]
    else:
            month = table_row.findAll('td')[0].text
            MoY.append((year + '-' + month ))
            Close.append(table_row.findAll('td')[3].text)

output_rows = pd.DataFrame()
output_rows['date'] = MoY
output_rows['wti'] = Close

In [29]:
output_rows = output_rows[0:12]
output_rows['record_date'] = str(date.today())
output_rows['website'] = 'longforecast'

output_rows

,date,wti,record_date,website
0,19-Oct,53.37,2019-10-21,longforecast
1,19-Nov,53.37,2019-10-21,longforecast
2,19-Dec,54.36,2019-10-21,longforecast
3,20-Jan,55.03,2019-10-21,longforecast
4,20-Feb,51.62,2019-10-21,longforecast
5,20-Mar,54.82,2019-10-21,longforecast
6,20-Apr,51.42,2019-10-21,longforecast
7,20-May,54.36,2019-10-21,longforecast
8,20-Jun,57.73,2019-10-21,longforecast
9,20-Jul,58.13,2019-10-21,longforecast


In [33]:
## need to switch back to Vteam to run this code
to_push = output_rows
# to_push.astype('str').to_sql('EP_WTI_archive', engine, if_exists='append', chunksize=50, index= None)
to_push.astype('str').to_sql('EP_WTI_archive', engine, if_exists='append', chunksize=50, index= None,schema='SCMAnalyticsDev.dbo')

In [ ]:
### this is another attempt to read the table from the PDF,
### but some how this didn't work out


# import PyPDF2

# # creating an object 
#   file = open('C:/Users/yangsh6/Desktop/Monthly_F_price/NEB/NEB_2019_02_21.pdf'', 'rb')

# # creating a pdf reader object
#   fileReader = PyPDF2.PdfFileReader(file)

# # print the number of pages in pdf file
#   print(fileReader.numPages)

# # select which page to extract
# pageObj = fileReader.getPage(0)

# # extract the text from that page
# pageObj.extractText()